In [1]:
import asyncio
from obniz import Obniz

async def onconnect(obniz):
    obniz.display.clear()
    obniz.display.print("Hello, World!")

obniz = Obniz('4542-8598')
obniz.onconnect = onconnect

asyncio.get_event_loop().run_forever()

RuntimeError: This event loop is already running

WebSocket connection is closed: code = 1000 (OK), no reason
_call_on_connect


In [7]:
import asyncio
from obniz import Obniz

hcsr04 = obniz.wired("HC-SR04", {"gnd":0, "echo":1, "trigger":2, "vcc":3})

async def onconnect(obniz):
    dist = hcsr04.measure()
    #obniz.display.clear()
    #obniz.display.print(dist)
    print(dist)


obniz = Obniz('4542-8598')
obniz.onconnect = onconnect

asyncio.get_event_loop().run_forever()

RuntimeError: This event loop is already running

WebSocket connection is closed: code = 1000 (OK), no reason


Task exception was never retrieved
future: <Task finished coro=<onconnect() done, defined at <ipython-input-7-0e133877c202>:6> exception=NameError("name 'a' is not defined",)>
Traceback (most recent call last):
  File "<ipython-input-7-0e133877c202>", line 7, in onconnect
    dist = hcsr04.measure(a)
NameError: name 'a' is not defined


_call_on_connect
WebSocket connection is closed: code = 1000 (OK), reason = obniz 45428598 was disconnected
WebSocket connection is closed: code = 1000 (OK), reason = obniz 45428598 was disconnected
WebSocket connection is closed: code = 1000 (OK), reason = obniz 45428598 was disconnected
WebSocket connection is closed: code = 1000 (OK), reason = obniz 45428598 was disconnected
WebSocket connection is closed: code = 1000 (OK), reason = obniz 45428598 was disconnected


In [1]:
import argparse
import logging
import time

import cv2
import numpy as np

from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

logger = logging.getLogger('TfPoseEstimator-WebCam')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

fps_time = 0


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='tf-pose-estimation realtime webcam')
    parser.add_argument('--camera', type=int, default=0)

    parser.add_argument('--resize', type=str, default='0x0',
                        help='if provided, resize images before they are processed. default=0x0, Recommends : 432x368 or 656x368 or 1312x736 ')
    parser.add_argument('--resize-out-ratio', type=float, default=4.0,
                        help='if provided, resize heatmaps before they are post-processed. default=1.0')

    parser.add_argument('--model', type=str, default='mobilenet_thin', help='cmu / mobilenet_thin / mobilenet_v2_large / mobilenet_v2_small')
    parser.add_argument('--show-process', type=bool, default=False,
                        help='for debug purpose, if enabled, speed for inference is dropped.')
    args = parser.parse_args()

    logger.debug('initialization %s : %s' % (args.model, get_graph_path(args.model)))
    w, h = model_wh(args.resize)
    if w > 0 and h > 0:
        e = TfPoseEstimator(get_graph_path(args.model), target_size=(w, h))
    else:
        e = TfPoseEstimator(get_graph_path(args.model), target_size=(432, 368))
    logger.debug('cam read+')
    cam = cv2.VideoCapture(args.camera)
    ret_val, image = cam.read()
    logger.info('cam image=%dx%d' % (image.shape[1], image.shape[0]))

    while True:
        ret_val, image = cam.read()

        logger.debug('image process+')
        humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=args.resize_out_ratio)

        logger.debug('postprocess+')
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

        logger.debug('show+')
        cv2.putText(image,
                    "FPS: %f" % (1.0 / (time.time() - fps_time)),
                    (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 0), 2)
        cv2.imshow('tf-pose-estimation result', image)
        fps_time = time.time()
        if cv2.waitKey(1) == 27:
            break
        logger.debug('finished+')

    cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'tf_pose'